<a href="https://colab.research.google.com/github/Jahid1990/Deep-Learning-Applications-R-Python/blob/master/Playing_around_different_custom_loss_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Some required packages

In [89]:
import pandas as pd
import keras
import numpy as np
from keras import layers
from keras.layers import Input, Dense,Conv1D,MaxPooling1D,Flatten,Concatenate
from keras.layers import TimeDistributed
from keras.models import Model
from keras import metrics
import keras.backend as K

# Dummy dataset creation

To have better intuition, lets create a small dataset having both discrete and continuous dependent variable(y_train). Making sure that dataset is applicable for all sort of loss functions we are going to explore.

In [90]:
x_train=np.random.rand(100,5)
y_train=np.concatenate((np.random.randint(0,2,(100,2)),np.random.rand(100,4)),axis=1)
print("shape of x_train:"+str(x_train.shape))
print("shape of y_train:"+str(y_train.shape))

shape of x_train:(100, 5)
shape of y_train:(100, 6)


In [94]:
print(" x_train:")
print(x_train[:3,:]) # first 3 rows of explanatory variables (x_train) 
print(" y_train:")
print(y_train[:3,:]) # first 3 rows of indenpendent variables (y_train)

 x_train:
[[0.58762746 0.49815876 0.06610818 0.51478354 0.58487054]
 [0.94996571 0.06768403 0.77131331 0.33419011 0.14821091]
 [0.42817061 0.32619562 0.52025564 0.72479127 0.51587583]]
 y_train:
[[0.         0.         0.47507594 0.1692404  0.47550315 0.82312835]
 [0.         0.         0.56238008 0.34338669 0.0205376  0.93589996]
 [0.         0.         0.74545935 0.41756426 0.62706896 0.80462387]]


In [208]:
def custom_act(x):
    e_x = 1/(1+tf.keras.backend.exp(-x))
    return e_x
x = np.array([[ 0.50839931,  0.49767588,  0.51260159]])
#softmax(x)
K.sigmoid(x_train)

<tf.Tensor: shape=(100, 5), dtype=float64, numpy=
array([[0.64282059, 0.62202654, 0.51652103, 0.62592718, 0.64218734],
       [0.72110828, 0.51691455, 0.68380492, 0.58277854, 0.53698505],
       [0.60543674, 0.58083343, 0.62720754, 0.67366121, 0.62618289],
       [0.60543924, 0.70727598, 0.66807792, 0.55530073, 0.55390834],
       [0.64108378, 0.71991134, 0.53439936, 0.69492569, 0.65378695],
       [0.69593637, 0.61883176, 0.53997433, 0.56129917, 0.53650323],
       [0.6134788 , 0.6221689 , 0.55542781, 0.71856159, 0.58802779],
       [0.56328009, 0.68836041, 0.68350647, 0.69822832, 0.60701771],
       [0.53382252, 0.64781493, 0.71070425, 0.64695805, 0.67060554],
       [0.68977872, 0.66410023, 0.70315767, 0.60368075, 0.50688958],
       [0.71321984, 0.59592568, 0.63209177, 0.71931907, 0.67383158],
       [0.59249422, 0.66196909, 0.67633444, 0.62810846, 0.65532387],
       [0.62068505, 0.66618083, 0.52321136, 0.55596442, 0.53031105],
       [0.65952364, 0.53986063, 0.66470711, 0.5893966

# Model architecture  
 Will define very simplistic model to have keep it simple for better understanding. 


In [222]:
input=Input(shape=(x_train.shape[1],)) # it should be equal to the number of columns of x_train

x=Dense(10,activation='relu')(input)
x=Dense(10,activation='tanh')(x)
out=Dense(2,activation=custom_act)(input)

model=Model(inputs=input,outputs=out)
model.summary()

Model: "functional_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
dense_96 (Dense)             (None, 2)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


# Binary Crossentropy

In [223]:
import tensorflow as tf 
def cus_binary_cross_entropy(y_true,y_pred):
  y_true=K.cast(y_true,'float32')
  y_pred=K.cast(y_pred,'float32')
  loss1=y_true[:,0]*K.log(y_pred[:,0]+K.epsilon())+(1-y_true[:,0])*K.log(1-y_pred[:,0]+K.epsilon())
  loss2=y_true[:,1]*K.log(y_pred[:,1]+K.epsilon())+(1-y_true[:,1])*K.log(1-y_pred[:,1]+K.epsilon())
  return -K.mean(loss1)

In [224]:
model.compile(loss=cus_binary_cross_entropy,optimizer='Adam',metrics=['accuracy'])
model.fit(x_train,y_train[:,:2], epochs=100,validation_split=.3,verbose=1,batch_size=100)

Epoch 1/100
1/1 [==============================] - 0s 141ms/step - loss: 0.7496 - accuracy: 0.5714 - val_loss: 0.7170 - val_accuracy: 0.6333
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 0.7490 - accuracy: 0.5714 - val_loss: 0.7167 - val_accuracy: 0.6333
Epoch 3/100
1/1 [==============================] - 0s 17ms/step - loss: 0.7485 - accuracy: 0.5714 - val_loss: 0.7164 - val_accuracy: 0.6333
Epoch 4/100
1/1 [==============================] - 0s 17ms/step - loss: 0.7479 - accuracy: 0.5714 - val_loss: 0.7162 - val_accuracy: 0.6333
Epoch 5/100
1/1 [==============================] - 0s 17ms/step - loss: 0.7474 - accuracy: 0.5714 - val_loss: 0.7159 - val_accuracy: 0.6333
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 0.7469 - accuracy: 0.5714 - val_loss: 0.7157 - val_accuracy: 0.6333
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 0.7463 - accuracy: 0.5714 - val_loss: 0.7154 - val_accuracy: 0.6333
Epoch 8/100
1/1 [==

In [166]:
x=np.array((.2,.6,.7,.8))
np.array((K.sigmoid(x[0]),K.sigmoid(x[1])))


array([0.549834  , 0.64565631])